In [13]:
import pandas as pd
from sklearn.manifold import TSNE
from pathlib import Path

In [38]:
file = Path("../public/data/anxiety_depression.csv")
df = pd.read_csv(file)
df

,Age,Gender,Education_Level,Employment_Status,Sleep_Hours,Physical_Activity_Hrs,Social_Support_Score,Anxiety_Score,Depression_Score,Stress_Level,...,Chronic_Illnesses,Medication_Use,Therapy,Meditation,Substance_Use,Financial_Stress,Work_Stress,Self_Esteem_Score,Life_Satisfaction_Score,Loneliness_Score
0,56,Male,Bachelor's,Unemployed,6.0,0.4,3,4,2,9,...,0,None,0,1,None,4,3,7,5,1
1,69,Female,Bachelor's,Retired,8.8,2.8,6,18,7,6,...,0,None,1,0,None,1,4,7,4,6
2,46,Female,Master's,Employed,5.3,1.6,5,5,13,8,...,0,None,0,1,None,8,7,8,1,1
3,32,Female,High School,Unemployed,8.8,0.5,4,6,3,4,...,1,None,0,0,None,7,4,8,4,4
4,60,Female,Bachelor's,Retired,7.2,0.7,2,7,15,3,...,0,None,1,1,Frequent,8,9,5,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,23,Female,Other,Student,5.1,0.1,7,16,15,5,...,0,Occasional,0,0,None,2,8,7,6,5
1196,43,Female,Master's,Employed,8.9,0.4,9,19,4,9,...,0,None,0,1,None,8,9,7,6,4
1197,71,Female,PhD,Retired,6.0,0.1,2,15,4,2,...,0,None,0,1,None,2,9,9,3,7
1198,21,Male,Other,Employed,8.5,2.7,8,12,20,2,...,1,None,0,0,None,8,5,1,3,7


In [39]:
def process(df):
    ndf = df.copy()
    cols = df.dtypes[df.dtypes == "object"].index.values
    for c in cols:
        vals = df[c].unique().tolist()
        ndf[c] = df[c].apply(lambda x: vals.index(x))

    return ndf.fillna(0)

ndf = process(df)
ndf

,Age,Gender,Education_Level,Employment_Status,Sleep_Hours,Physical_Activity_Hrs,Social_Support_Score,Anxiety_Score,Depression_Score,Stress_Level,...,Chronic_Illnesses,Medication_Use,Therapy,Meditation,Substance_Use,Financial_Stress,Work_Stress,Self_Esteem_Score,Life_Satisfaction_Score,Loneliness_Score
0,56,0,0,0,6.0,0.4,3,4,2,9,...,0,0,0,1,0,4,3,7,5,1
1,69,1,0,1,8.8,2.8,6,18,7,6,...,0,0,1,0,0,1,4,7,4,6
2,46,1,1,2,5.3,1.6,5,5,13,8,...,0,0,0,1,0,8,7,8,1,1
3,32,1,2,0,8.8,0.5,4,6,3,4,...,1,0,0,0,0,7,4,8,4,4
4,60,1,0,1,7.2,0.7,2,7,15,3,...,0,0,1,1,1,8,9,5,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,23,1,3,3,5.1,0.1,7,16,15,5,...,0,1,0,0,0,2,8,7,6,5
1196,43,1,1,2,8.9,0.4,9,19,4,9,...,0,0,0,1,0,8,9,7,6,4
1197,71,1,4,1,6.0,0.1,2,15,4,2,...,0,0,0,1,0,2,9,9,3,7
1198,21,0,3,2,8.5,2.7,8,12,20,2,...,1,0,0,0,0,8,5,1,3,7


In [40]:
def project(df):
    tsne = TSNE(n_components=2)
    result = tsne.fit_transform(df.to_numpy())
    return result

coords = project(ndf)
df["x"] = coords[:,0]
df["y"] = coords[:,1]
df.to_csv(file.with_suffix(".dr.csv"), index=False)